In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import matplotlib.pyplot as plt

In [ ]:
train_range = 28
test_range = 7
batch_size = 32
learning_rate = 0.001

In [ ]:
dim_l = 31 # seasonal model, 24 hours + 7 days
dim_z = 1
initial_variance = 20

In [ ]:
all_z = np.load('data/formatted_traffic.npy')
train_z = np.reshape(all_z[:,:train_range,:], [all_z.shape[0], -1])
test_z = np.reshape(all_z[:,train_range:,:], [all_z.shape[0], -1])
del all_z
train_x = np.load('data/train_features.npy')
test_x = np.load('data/test_features.npy')
train_x = np.transpose(train_x, (1,0,2))
test_x = np.transpose(test_x, (1,0,2))

In [ ]:
print(train_z.shape)
print(test_z.shape)
print(train_x.shape)
print(test_x.shape)

In [ ]:
tf.enable_eager_execution()
def fn(params, inputs):
    a = params
    b = inputs

    return tf.add(a,b)

inputs = np.array([x for x in range(27)]).reshape([9,3]).astype(np.float32)
print(inputs.shape)
inputs = tf.tile(tf.expand_dims(tf.convert_to_tensor(inputs),0), (4,1,1))
dummy_a = tf.ones([4,3])
print(inputs.shape)
forward_states = tf.scan(fn, tf.transpose(inputs, [1,0,2]),
                        initializer = (dummy_a))


In [ ]:
forward_states.shape

In [ ]:
print(tf.expand_dims(inputs,-1).shape)
print(dummy_a.shape)
print(forward_states.shape)

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    dim_l = 2
    dim_z = 1
    sample_len = 25
    
    z = np.array([x+np.random.randn() for x in range(sample_len)]).astype(np.float32)
    z = tf.expand_dims(tf.expand_dims(tf.convert_to_tensor(z), -1),0)
    l_0 = tf.expand_dims(tf.expand_dims(tf.constant([0,0], dtype=tf.float32), -1), 0)
    P_0 = tf.expand_dims(tf.constant([[5,5],[5,5]], dtype = tf.float32), 0)
    F = tf.expand_dims(tf.tile(tf.expand_dims(tf.convert_to_tensor(np.array([[1,0],[0,1]], dtype = np.float32)), 0), (sample_len,1,1)),0)
    Q = tf.expand_dims(tf.random.normal((sample_len,dim_l, dim_l)), 0)
    R = tf.expand_dims(tf.random.normal((sample_len, dim_z, dim_z)),0)
    a = tf.expand_dims(tf.ones((sample_len, dim_l, dim_z)),0)
    b = tf.expand_dims(tf.ones((sample_len,dim_z)),0)

    kf = KalmanFilter(dim_l=dim_l, dim_z=dim_z, l_0=l_0, P_0=P_0, F=F, a=a, b=b, Q=Q, R=R, z=z)
    sess.run(tf.global_variables_initializer())
    
    l_filtered, P_filtered = kf.filter()
    a = tf.squeeze(a,0)
    b = tf.squeeze(b,0)
    l_filtered = tf.squeeze(l_filtered, 0)
    preds = []
    for i in range(z.shape[1]):
        preds.append((tf.matmul(a[i],l_filtered[i], transpose_a=True)+b[i]).eval())
    preds = np.concatenate(np.concatenate(preds))
    plt.plot(preds)
    plt.show()

In [ ]:
dim_l = 2
dim_z = 1

tf.reset_default_graph()
z = np.array([x for x in range(27)]).astype(np.float32)
z = tf.expand_dims(tf.expand_dims(tf.convert_to_tensor(z), -1),0)
l_0 = tf.expand_dims(tf.expand_dims(tf.constant([0,0], dtype=tf.float32), -1), 0)
P_0 = tf.expand_dims(tf.constant([[5,5],[5,5]], dtype = tf.float32), 0)
F = tf.expand_dims(tf.tile(tf.expand_dims(tf.convert_to_tensor(np.array([[1,0],[0,1]], dtype = np.float32)), 0), (27,1,1)),0)
Q = tf.expand_dims(tf.random.normal((27,dim_l, dim_l)), 0)
R = tf.expand_dims(tf.random.normal((27, dim_z, dim_z)),0)
a = tf.expand_dims(tf.ones((27, dim_l, dim_z)),0)
b = tf.expand_dims(tf.ones((27,dim_z)),0)

kf = KalmanFilter(dim_l=dim_l, dim_z=dim_z, l_0=l_0, P_0=P_0, F=F, a=a, b=b, Q=Q, R=R, z=z)
l_filtered, P_filtered = kf.filter()
a = tf.squeeze(a,0)
b = tf.squeeze(b,0)
l_filtered = tf.squeeze(l_filtered, 0)
preds = []
for i in range(z.shape[1]):
    preds.append(tf.matmul(a[i],l_filtered[i], transpose_a=True)+b[i])

In [ ]:
kf = KalmanFilter(dim_l=dim_l, dim_z=dim_z, l_0=l_0, P_0=P_0, F=F, a=a, b=b, Q=Q, R=R, z=z)
l_filtered, P_filtered = kf.filter()

In [ ]:
l_filtered

### Testing conversion of LSTM output to SSM parameters

In [17]:
class KalmanFilter(object):
    """
    This class defines a kalman filter

    l - latent state
    l_a_priori - A priori state estimate
    l_a_posteriori - A posteriori state estimate

    P_a_priori - A priori error covariance
    P_a_posteriori - A posteriori error covariance

    F - state-transition model
    Q - covariance of the process noise    
    a, b - observation model and bias
    R - covariance of the observation noise
    z - observation

    y_pre - measurement pre-fit residual
    S - Pre-fit residual covariance
    K - Kalman gain
    y_post - measurement post-fit residual
    """
    
    def __init__(self, dim_l, dim_z, **kwargs):
        self.dim_l = dim_l
        self.dim_z = dim_z
        self.dim_y = dim_z

        # lambda initializer for identity matrices
        self.eye_init = lambda shape, dtype = np.float32: np.eye(*shape, dtype = dtype)

        self._I = tf.constant(self.eye_init((dim_l, dim_l)), name= 'I')

        # Pop all variables
        '''This section requires these kwargs to exist, cannot handle missing args'''
        with tf.variable_scope('KF', reuse = tf.AUTO_REUSE):
            init = kwargs.pop('l_0', np.zeros((dim_l, ), dtype=np.float32))
            self.l_0 = tf.get_variable('l_0', initializer=init, trainable=False)  # Predicted state estimate

            init = kwargs.pop('P_0', self.eye_init((dim_l, dim_l)))
            self.P_0 = tf.get_variable('P_0', initializer=init, trainable=False)  # Predicted error covariance

            init = kwargs.pop('F', self.eye_init((dim_l, dim_l)))
            self.F = tf.get_variable('F', initializer = init)

            init = kwargs.pop('Q', self.eye_init((dim_l, dim_l)))
            self.Q = tf.get_variable('Q', initializer = init, trainable = False)

            init = kwargs.pop('a', np.zeros((dim_l,), dtype=np.float32))
            self.a = tf.get_variable('a', initializer=init)

            init = kwargs.pop('b', np.zeros((dim_z,), dtype=np.float32))
            self.b = tf.get_variable('b', initializer=init)

            init = kwargs.pop('R', self.eye_init((dim_z, dim_z)))
            self.R = tf.get_variable('R', initializer=init, trainable = False)

            init = kwargs.pop('y_0', np.zeros((self.dim_y), dtype = np.float32))
            self.y_0 = tf.get_variable('y_0', initializer = init) # initial prediction

#         print('l_0', self.l_0)
#         print('P_0', self.P_0)
#         print('F', self.F)
#         print('Q', self.Q)
#         print('a', self.a)
#         print('b', self.b)
#         print('R', self.R)

        '''This section can handle missing kwargs'''
        self.z = kwargs.pop('z', None)
        if self.z is None:
            self.z = tf.placeholder(tf.float32, shape = [None, None, dim_z], name = 'z')
#         print('z', self.z)

    def forward_step_fn(self, params, inputs):
        """
        Forward step over a batch
        params contains: l_a_posteriori, P_a_posteriori, TODO: What else??
        inputs contains: batch_size, variable dimensions, TODO: Need this??

        Calculates prior distributions based on the given posterior distributions and the current residual
                updates posterior distributions based on the new prior distributions
        """
        '''Shapes:
            z = (bs, dim_z)
            l_a_posteriori = (bs, dim_l, dim_z)
            P_a_posteriori = (bs, dim_l, dim_l)
            F = (bs, dim_l, dim_l)
            Q = (bs, dim_l, dim_l)
            R = (bs, dim_z, dim_z)
            a = (bs, dim_l, dim_z)
            b = (bs, dim_z)
        '''
        
        z, F, Q, R, a, b = inputs
        l_a_posteriori, P_a_posteriori, y_pre = params

#         print('z',z)
#         print('F', F)
#         print('Q', Q)
#         print('R', R)
#         print('a', a)
#         print('b', b)
#         print('l', l_a_posteriori)
#         print('P', P_a_posteriori)
#         print('y_pre', y_pre)
        
        l_a_priori = tf.matmul(F,l_a_posteriori)
        P_a_priori = tf.matmul(tf.matmul(F,P_a_posteriori), F, transpose_b = True) + Q
        
        y_pre = tf.expand_dims(z - tf.add(tf.squeeze(tf.matmul(a, l_a_priori, transpose_a=True),-1), b),-1)
#         print('y_pre', y_pre)
        S = R + tf.matmul(tf.matmul(a, P_a_priori, transpose_a=True), a)
#         print('S', S)
#         S_inv = tf.matrix_inverse(S)
        S_inv = tf.reciprocal(S)
        K = tf.matmul(tf.matmul(P_a_priori, a), S_inv)
        
        l_a_posteriori = l_a_priori + tf.matmul(K,y_pre)

        I_Ka = self._I-tf.matmul(K,a, transpose_b=True)
        P_a_posteriori = tf.matmul(tf.matmul(I_Ka, P_a_priori), I_Ka, transpose_b=True) + \
                         tf.matmul(tf.matmul(K,R), K, transpose_b=True)
        
        y_post = z - tf.add(tf.squeeze(tf.matmul(a, l_a_posteriori, transpose_a=True),-1), b)
        return l_a_posteriori, P_a_posteriori, y_post

    def compute_forwards(self):
        """
        Compute the forward step in Kalman Filter
        The forward pass is initialized with p(x_1) = N(self.x, self.P)
        We return the mean and covariance for p(x_t|x_tm1) for t=2, ..., T+1
        and the filtering distribution p(x_t|z_1:t) for t=1, ..., T
        """
        def trans(tensor):
            if len(tensor.shape)==3:
                return tf.transpose(tensor, [1,0,2])
            else:
                return tf.transpose(tensor, [1,0,2,3])
            
        forward_states = tf.scan(self.forward_step_fn,
                                 elems = (trans(self.z),trans(self.F),
                                          trans(self.Q),trans(self.R),
                                          trans(self.a),trans(self.b)),
                                initializer=(self.l_0, self.P_0, self.y_0))
        
        return forward_states
    
    def Kfilter(self):
        l_filtered, P_filtered, y_preds = self.compute_forwards()
        return (tf.transpose(l_filtered, [1,0,2,3]),
                tf.transpose(P_filtered, [1,0,2,3]),
                tf.transpose(y_preds, [1,0,2]))
        

In [20]:
class LSTM_SSM_model(object):
    def __init__(self, sess):
        self.sess = sess
        self.saved_model_location = './tmp/LSTM_SSM_model.ckpt'
#         self.saved_model_location=''
        self.saver = None
        
        self.learning_rate = 0.001
        self.batch_size = 32
        self.train_range = 28
        num_samples, sample_len, feature_len = (963,672,994)
        
        self.lstm_sizes = [128,64]
        last_lstm = self.lstm_sizes[-1]
        
        self.dim_l = 31 # seasonal model, 24 hours + 7 days
        self.dim_z = 1
        self.initial_variance = 20
        
        self.F = tf.tile(tf.expand_dims(tf.expand_dims(tf.eye(self.dim_l),0),0),
                         (self.batch_size,sample_len,1,1))
        with tf.variable_scope('KF', reuse = tf.AUTO_REUSE):
            # Initialize with random values
            #self.a = tf.placeholder(tf.float32, shape = [None, None, self.dim_l, self.dim_z], name = 'a')
            self.a = tf.get_variable(initializer = tf.ones([self.batch_size, sample_len, self.dim_l, self.dim_z]),
                                     dtype = tf.float32, name = 'a')
            self.W_a = tf.get_variable(initializer = tf.random.normal([self.batch_size, sample_len, self.dim_l, last_lstm]),
                                       dtype = tf.float32, name = 'W_a') # dim_l, lstm_outputs.shape[-1]
            self.bias_a = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, self.dim_l, self.dim_z]),
                                          dtype = tf.float32, name = 'bias_a')

            # Initialize with random values
            #self.b = tf.placeholder(tf.float32, shape = [None, None, self.dim_z], name = 'b')
            self.b = tf.get_variable(initializer = tf.ones([self.batch_size, sample_len, self.dim_z]),
                                    dtype = tf.float32, name = 'b')
            self.W_b = tf.get_variable(initializer = tf.random.normal([self.batch_size, sample_len,self.dim_z, last_lstm]),
                                       dtype = tf.float32, name = 'W_b') # dim_z, lstm_outputs.shape[-1]
            self.bias_b = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, self.dim_z]),
                                          dtype = tf.float32, name = 'bias_b')

            #self.Q = tf.placeholder(tf.float32, shape = [None, None, self.dim_l, self.dim_l], name = 'Q')
            self.Q = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, self.dim_l, self.dim_l]),
                                     dtype = tf.float32, name = 'Q')
            self.W_Q = tf.get_variable(initializer =tf.random.normal([self.batch_size, sample_len, self.dim_l, last_lstm]),
                                       dtype = tf.float32, name = 'W_Q')
            self.bias_Q = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, self.dim_l]),
                                          dtype = tf.float32, name = 'bias_Q')

            #self.R = tf.placeholder(tf.float32, shape = [None, None, self.dim_z], name = 'R')
            self.R = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, self.dim_z, self.dim_z]),
                                     dtype = tf.float32, name = 'R')
            self.W_R = tf.get_variable(initializer = tf.random.normal([self.batch_size, sample_len, self.dim_z, last_lstm]),
                                       dtype = tf.float32, name = 'W_R')
            self.bias_R = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, self.dim_z]),
                                          dtype = tf.float32, name = 'bias_R')
        
        self.l_0 = tf.zeros([self.batch_size, self.dim_l, self.dim_z], dtype = tf.float32)
        self.P_0 = tf.tile(tf.expand_dims(self.initial_variance*tf.eye(self.dim_l, dtype = tf.float32),0),
                           (self.batch_size, 1, 1))
        self.y_0 = tf.zeros([self.batch_size, self.dim_z], dtype = tf.float32)
        
        self.lstm_input = tf.placeholder(tf.float32, shape= [None, sample_len, feature_len], name = 'lstm_input')
        self.lstm_output = tf.get_variable(initializer = tf.zeros([self.batch_size, sample_len, last_lstm]),
                                           dtype = tf.float32, name = 'lstm_output')
        self.z = tf.placeholder(tf.float32, shape = [None, sample_len, self.dim_z], name = 'z')
        
        self.losses = []

    def load_data(self):
        print("Loading Data...")
        # load data
        all_z = np.load('data/formatted_traffic.npy')
        self.train_z = np.reshape(all_z[:,:self.train_range,:], [all_z.shape[0], -1, 1])
        self.test_z = np.reshape(all_z[:,self.train_range:,:], [all_z.shape[0], -1])
        del all_z
        train_x = np.load('data/train_features.npy')
        test_x = np.load('data/test_features.npy')
        self.train_x = np.transpose(train_x, (1,0,2))
        self.test_x = np.transpose(test_x, (1,0,2))
        print("...Data Loaded")
        return self
        
    def build_LSTM(self):
        with tf.variable_scope('LSTM', reuse=tf.AUTO_REUSE):
            
            lstms = [tf.contrib.rnn.LSTMCell(size) for size in self.lstm_sizes]
            dropouts = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = 0.8) for lstm in lstms]

            cell = tf.contrib.rnn.MultiRNNCell(dropouts)
            initial_state = cell.zero_state(self.batch_size, tf.float32)

            self.lstm_output, final_state = tf.nn.dynamic_rnn(cell, self.lstm_input, initial_state = initial_state)

        return self
    
    def affine_transformations(self):
#         with tf.variable_scope('affine_transformations'):
            
        self.lstm_output = tf.expand_dims(self.lstm_output, -1)

        self.a = tf.expand_dims(tf.squeeze(tf.matmul(self.W_a, self.lstm_output)),-1)
        self.a = self.a + self.bias_a

        self.b = tf.expand_dims(tf.squeeze(tf.matmul(self.W_b, self.lstm_output)),-1)
        self.b = self.b + self.bias_b

        transition_error = tf.squeeze(tf.matmul(self.W_Q, self.lstm_output))
        transition_error = transition_error + self.bias_Q
        self.Q = tf.linalg.diag(transition_error)

        observation_error = tf.expand_dims(tf.squeeze(tf.matmul(self.W_R, self.lstm_output)),-1)
        observation_error = observation_error + self.bias_R
        self.R = tf.linalg.diag(observation_error)
        return self
    
    def build_model(self):
        self.kf = KalmanFilter(dim_l=self.dim_l,
                               dim_z=self.dim_z,
                               l_0 = self.l_0,
                               P_0 = self.P_0,
                               F = self.F,
                               Q = self.Q,
                               a = self.a,
                               b = self.b,
                               R = self.R,
                               z = self.z,
                               y_0 = self.y_0
                              )
        with tf.variable_scope('Kalman_Filter'):
            self.l_filtered, self.P_filtered, self.y_preds = self.kf.Kfilter()

        return self
    
    def build_loss(self):
        # some series of equations
        '''Useful shapes(Ideally):
            l_a_posteriori(batch) - (batch_size, sample_len, dim_l)
            P_a_posteriori(batch) - (batch_size, sample_len, dim_l,dim_l)
            
            inputs:
                mu_0, a, F, l_a_posteriori?
                Sigma_0, a, R, F, P_a_posteriori, Q
        '''
        with tf.variable_scope('loss'):
            self.loss = tf.losses.mean_squared_error(self.z, self.y_preds)
        
        # What is the tensorflow method of recursively adding values to tensor?
        # Iterate through range(sample_len), apply functions, then tf.stack() the elements together in the end?
        # tf.scan() ??
        
        
        
#         self.loss = tf.losses.mean_squared_error(self.z, self.predictions)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        return self
    
    def initialize_variables(self):
        self.saver = tf.train.Saver()
        
        if self.saved_model_location is not '' and os.path.exists(self.saved_model_location):
            print("Restoring model from {}".format(self.saved_model_location))
            self.saver.restore(self.sess, self.saved_model_location)
        else:
            self.sess.run(tf.global_variables_initializer())
        return self
    
    def train(self, epochs):
        start = time.time()
        for i in range(epochs):
            epoch_loss = []
            perm = np.random.permutation(self.train_x.shape[0])
            for idx in range((self.train_x.shape[0]//self.batch_size)):
                slc = np.array([perm[i] for i in range(idx*32, (idx+1)*32)])
                feed_dict = {self.lstm_input: self.train_x[slc],
                            self.z: self.train_z[slc]}
                loss_, _ = self.sess.run([self.loss, self.optimizer], feed_dict=feed_dict)
                epoch_loss.append(loss_)
            epoch_loss = np.mean(epoch_loss)
            self.losses.append(epoch_loss)
            print("Epoch #{}\tTime Elapsed: {}\tLoss {}".format(i, (time.time()-start)/60, epoch_loss))
        
        self.saver.save(self.sess, self.saved_model_location)
        print("Model Saved")
        return self.losses

In [21]:
tf.reset_default_graph()

def initialize_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]

    print([str(i.name) for i in not_initialized_vars]) # only for testing
    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))

with tf.Session() as sess:
    
    model = LSTM_SSM_model(sess)
    model.build_LSTM().affine_transformations().build_model().build_loss().load_data()
#     initialize_uninitialized(sess)
    model.initialize_variables()
    model.train(epochs = 1)
#     feed_dict = {model.lstm_input:model.train_x[:32],
#                 model.z: model.train_z[:32]}
#     preds = model.sess.run([model.y_preds], feed_dict=feed_dict)
#     preds

Loading Data...
...Data Loaded
Epoch #0	Time Elapsed: 1.0416364590326945	Loss nan
Model Saved


### Working example of just LSTM

lstm_inputs: (batch_size, 672, 994)

lstm_outputs: (batch_size, 672, 64)

In [ ]:
class LSTM_model(object):
    def __init__(self, sess):
        self.sess = sess
        self.saved_model_location = './tmp/LSTM_model.ckpt'
        
        self.learning_rate = 0.001
        self.batch_size = 32
        self.train_range = 28
        train_x_shape = [963,672,994]
        train_z_shape = [963,672]
    
        self.lstm_input = tf.placeholder(tf.float32, shape= [None, train_x_shape[1], train_x_shape[2]])
        self.lstm_output = tf.placeholder(tf.float32, shape = [None, train_x_shape[1]])
        self.labels_ = tf.placeholder(tf.float32, shape = [None, train_z_shape[1]])
        
        self.saver = None
        self.losses = []
    
    def load_train_data(self):
        print("Loading data...")
        # load data
        all_z = np.load('data/formatted_traffic.npy')
        self.train_z = np.reshape(all_z[:,:self.train_range,:], [all_z.shape[0], -1])
        self.test_z = np.reshape(all_z[:,self.train_range:,:], [all_z.shape[0], -1])
        del all_z
        train_x = np.load('data/train_features.npy')
        test_x = np.load('data/test_features.npy')
        self.train_x = np.transpose(train_x, (1,0,2))
        self.test_x = np.transpose(test_x, (1,0,2))
        print("...Data loaded")
        return self
    
    def build_LSTM(self):
        lstms = [tf.contrib.rnn.LSTMCell(size, reuse = tf.AUTO_REUSE) for size in [128,64]]
        dropouts = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = 0.8) for lstm in lstms]

        cell = tf.contrib.rnn.MultiRNNCell(dropouts)
        initial_state = cell.zero_state(self.batch_size, tf.float32)

        self.lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cell, self.lstm_input, initial_state = initial_state)

        self.predictions = tf.squeeze(tf.contrib.layers.fully_connected(self.lstm_outputs, 1, activation_fn=tf.sigmoid))
        return self
    
    def build_loss(self):
        self.loss = tf.losses.mean_squared_error(self.labels_, self.predictions)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        return self
    
    def initialize_variables(self):
        self.saver = tf.train.Saver()
        
        if self.saved_model_location is not '' and os.path.exists(self.saved_model_location):
            print("Restoring model from {}".format(self.saved_model_location))
            self.saver.restore(self.sess, self.saved_model_location)
        else:
            self.sess.run(tf.global_variables_initializer())
        
        return self
    
    def train(self, epochs):
        start = time.time()
        for i in range(epochs):
            epoch_loss = []
            perm = np.random.permutation(self.train_x.shape[0])
            for idx in range((self.train_x.shape[0]//self.batch_size)):
                slc = np.array([perm[i] for i in range(idx*32, (idx+1)*32)])
                feed_dict = {self.lstm_input: self.train_x[slc],
                            self.labels_: self.train_z[slc]}
                loss_, _ = self.sess.run([self.loss, self.optimizer], feed_dict=feed_dict)
                epoch_loss.append(loss_)
            epoch_loss = np.mean(epoch_loss)
            self.losses.append(epoch_loss)
            print("Epoch #{}\tTime Elapsed: {}\tLoss {}".format(i, (time.time()-start)/60, epoch_loss))
            
        self.saver.save(self.sess, self.saved_model_location)
        print("Model Saved")
        return self.losses
    
    def test(self):
        feed_dict = {self.lstm_input: self.train_x[:32],
                    self.labels_: self.train_z[:32]}
        preds = self.sess.run([self.predictions], feed_dict=feed_dict)
        return preds

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = LSTM_model(sess)
    model.build_LSTM().build_loss().initialize_variables().load_train_data()
    losses = model.train(epochs = 1)
    plt.plot(losses)
    plt.show()

In [ ]:
with tf.Session() as sess:
    model = LSTM_model(sess)
    model.build_LSTM().initialize_variables().load_train_data()
    preds = model.test()
    plt.plot(preds[0][0])
    plt.show()